In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append("../../") 

In [ ]:
import pandas as pd
import numpy as np
import numpyro
numpyro.set_host_device_count(4)

from epimodel import preprocess_data, run_model, EpidemiologicalParameters
from epimodel.models.models import default_model

In [ ]:
data = preprocess_data('../../data/all_merged_data_2021-01-22.csv')
data.featurize() 
data.mask_new_variant(new_variant_fraction_fname='../../data/nuts3_new_variant_fraction.csv')
data.mask_from_date('2021-01-09') 

In [ ]:
ep.DPD[0, :12].sum()

In [ ]:
data.Ds[161+5]

In [ ]:
data.Ds[7]

In [ ]:
np.arange(10)[:6]

In [ ]:
ep.DPD[0, :33].sum()

In [ ]:
ep.DPC[0, :15].sum()

In [ ]:
ep = EpidemiologicalParameters()

In [ ]:
import yaml

In [ ]:
info = yaml.safe_load(open('default_model-16-03;08-19-21.yaml', 'r'))

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import numpy as np

In [ ]:
plt.title('Warmup Treepdepth')
plt.plot(np.log2(info['warmup']['num_steps'][251:500]))
plt.plot(np.log2(info['warmup']['num_steps'][:250]))
plt.plot(np.log2(info['warmup']['num_steps'][501:750]))
plt.plot(np.log2(info['warmup']['num_steps'][751:]))

In [ ]:
plt.title('Sample Treepdepth')
plt.plot(np.log2(info['sample']['num_steps'][:1250]))
plt.plot(np.log2(info['sample']['num_steps'][1251:2500]))
plt.plot(np.log2(info['sample']['num_steps'][2501:3750]))
plt.plot(np.log2(info['sample']['num_steps'][3751:]))

In [ ]:
samples, warmup_samples, info, mcmc = run_model(default_model, data, ep, num_samples=25, target_accept=0.8, num_warmup=25, num_chains=2, max_tree_depth=15)

In [ ]:
info['warmup']['adapt_state'].inverse_mass_matrix.shape

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
info['warmup']['adapt_state'].inverse_mass_matrix[-1, :]

In [ ]:
1/np.sqrt(info['warmup']['adapt_state'].inverse_mass_matrix[-1, :])

In [ ]:
info['warmup']['adapt_state'].mass_matrix_sqrt[-1, :]

In [ ]:
1./np.sqrt(info['warmup']['adapt_state'].inverse_mass_matrix[-1, :])

In [ ]:
np.sqrt(info['warmup']['adapt_state'].inverse_mass_matrix[-1, :])

In [ ]:
info['warmup']['adapt_state'].mass_matrix_sqrt_inv[-1, :]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(np.log2(np.array(info['sample']['num_steps'])+1))

In [ ]:
plt.plot(np.log2(np.array(info['warmup']['num_steps'])+1))

In [ ]:
iar_t.shape

In [ ]:
data.nDs

In [ ]:
info['sample']

In [ ]:
import arviz as az

In [ ]:
inf_data = az.from_netcdf("../default_res.netcdf")

In [ ]:
ess = az.ess(inf_data)

In [ ]:
ess.alpha_i

In [ ]:
samples

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(3, 3), dpi=300)
plt.plot(np.log2(np.array(info['warmup']['num_steps'])+1), label='warmup')
plt.plot(np.log2(np.array(info['sample']['num_steps'])+1), label='sample')
plt.legend()

In [ ]:
plt.plot(np.sort(mcmc._warmup_state.adapt_state.inverse_mass_matrix))
plt.yscale('log')

In [ ]:
from numpyro.infer import MCMC, NUTS, init_to_median
from jax import random
import jax

nuts_kernel2 = NUTS(
    default_model,
    init_strategy=init_to_median,
    target_accept_prob=0.9,
    max_tree_depth=20,
)
mcmc2 = MCMC(
    nuts_kernel2,
    num_samples=250,
    num_warmup=250,
    num_chains=1,
    chain_method="parallel",
)

In [ ]:
latent_dict = hmcstate.z

In [ ]:
hmcstate.adapt_state

In [ ]:
rng_key = random.PRNGKey(0)
hmcstate = nuts_kernel2.init(rng_key, 250, model_args=(data, ep))
latent_dict = hmcstate.z
z_flat, unravel_fn = jax.flatten_util.ravel_pytree(latent_dict)
inverse_mass_matrix = unravel_fn(info['warmup']['adapt_state'].inverse_mass_matrix[-1, :])

In [ ]:
keys = list(inverse_mass_matrix.keys())
mean_key_vals = [np.mean(inverse_mass_matrix[k]) for k in keys]
sorted_keys = sorted(keys, key=lambda k: mean_key_vals[keys.index(k)])

In [ ]:
import matplotlib.pyplot as plt 

import seaborn as sns
plt.figure(figsize=(4, 3), dpi=300)
for k in sorted_keys:
    m = inverse_mass_matrix[k].flatten()
    if m.size > 1:
        plt.plot(1+np.arange(m.size), np.sort(m), label=k)
    else:
        plt.scatter(1, m, label=k, s=60, marker='*', zorder=5)
    plt.yscale('log')
    plt.xscale('log')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.xlim([0.8, 10**4.5])
plt.title('Inverse Mass Matrix')
sns.despine()

Okay, this I think we are gonna have to pretty much live with. 

# Take a look at results. 

In [ ]:
plt.plot(np.sort(samples['seeding'].reshape((250, 114)).var(axis=0)))
plt.yscale('log')

In [ ]:
alt_seeding = np.log(samples['seeding']*10)
plt.plot(np.sort(alt_seeding.reshape((250, 114)).var(axis=0)))
plt.yscale('log')

In [ ]:
for r_i in data.C_indices[2]:
    print(r_i)

In [ ]:
len(data.Rs)

In [ ]:
data.C_indices[0]

In [ ]:
ep.generate_pmf_statistics_str(ep.DPC)

In [ ]:
ep.generate_pmf_statistics_str(ep.DPD)

In [ ]:
self.infection_to_fatality_delays = {
                "England": {
                    "mean": 21.82,
                    "disp": 14.26,
                    "dist": "negbinom",
                },

In [ ]:
var = 10.92 + 10.93**2/5.41

In [ ]:
np.sqrt(var)

In [ ]:
ep.DPC

In [ ]:
from epimodel.plotting.intervention_plots import plot_intervention_effectiveness, plot_intervention_correlation, combine_npi_samples

In [ ]:
plot_intervention_effectiveness(samples, data.CMs, xlim=[-25, 25])

In [ ]:
plot_intervention_correlation(samples, data.CMs)

In [ ]:
cols = list(range(10))

In [ ]:
main_panel_dict = {
    'Nightclubs Closed': {
        'npis': ['Some Face-to-Face Businesses Closed'],
        'type': "exclude",
        'color': cols[0],
    },
    'Gastronomy Closed': {
        'npis': ['Gastronomy Closed'],
        'type': 'exclude',
        'color': cols[0],
    },
    'Leisure Venues Closed': {
        'npis': ['Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
    },
    'Retail Closed': {
        'npis': ['Retail Closed'],
        'type': 'exclude',
        'color': cols[0],
    },
    'All Nonessential\nBusinesses Closed': {
        'npis': ['Retail Closed', 'Some Face-to-Face Businesses Closed', 
                'Gastronomy Closed', 'Leisure Venues Closed'],
        'type': 'exclude',
        'color': cols[0],
    },
    'All Gatherings Banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Gatherings limited to 2': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
    },
    'Gatherings limited to 6 from 2 Households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Gatherings limited to 6': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Gatherings limited to 30': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'All Public Gatherings Banned': {
        'npis': ['Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
#                  'Private Indoor Gathering Person Limit - 1',
#                  'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 6',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Public Gatherings limited to 2': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
#                  'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 6',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
    },
    'Public Gatherings limited to 6 from 2 Households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 6',
#                  'Private Indoor Gathering Person Limit - 30',
#                  'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Public Gatherings limited to 6': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
#                  'Private Indoor Gathering Person Limit - 6',
#                  'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Public Gatherings limited to 30': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 6',
                 'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 6',
#                  'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'All Private Gatherings Banned': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
#                  'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 6',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                 'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Private Gatherings limited to 2': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
#                  'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 6',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        "type": "exclude",
        'color': cols[3],
    },
    'Private Gatherings limited to 6 from 2 Households': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 6',
#                  'Public Indoor Gathering Person Limit - 30',
#                  'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Private Gatherings limited to 6': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
#                  'Public Indoor Gathering Person Limit - 6',
#                  'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'Private Gatherings limited to 30': {
        'npis': [#'Public Indoor Gathering Person Limit - 1',
                 #'Public Indoor Gathering Person Limit - 2',
                 #'Public Indoor Gathering Person Limit - 6',
#                  'Public Indoor Gathering Person Limit - 30',
                 #'Extra Public Indoor Household Limit',
                 #'Private Indoor Gathering Person Limit - 1',
                 #'Private Indoor Gathering Person Limit - 2',
                 #'Private Indoor Gathering Person Limit - 6',
                 'Private Indoor Gathering Person Limit - 30',
                 #'Extra Private Indoor Household Limit'
                ],
        'type': 'exclude',
        'color': cols[3],
    },
    'All Educational\nInstitutions Closed': {
        'npis': ['Primary Schools Closed', 'Secondary Schools Closed', 'Universities Away'],
        'type': 'exclude',
        'color': cols[2],
    },
    'Curfew': {
        'npis': ['Curfew'],
        'type': 'exclude',
        'color': cols[2]
    },
    'Mandatory Mask\nWearing': {
        'npis': ['Mandatory Mask Wearing >= 3'],
        'type': 'exclude',
        'color': cols[2],
    },
 }

npi_names = main_panel_dict.keys()
comb_cm_effects, new_names = combine_npi_samples(main_panel_dict, samples['alpha_i'], data.CMs)
import matplotlib.pyplot as plt
plt.figure(figsize=(6, 8), dpi=300)
plot_intervention_effectiveness(comb_cm_effects, new_names, xlim=[-20, 50], newfig=False)
plt.grid(axis='x')
plt.title('Drop Out')
plt.savefig('dropout.pdf', bbox_inches='tight')

In [ ]:

import matplotlib.pyplot as plt
plt.figure(figsize=(6, 8), dpi=300)
plot_intervention_effectiveness(comb_cm_effects, new_names, xlim=[-20, 50], newfig=False)
plt.grid(axis='x')
plt.title('Drop Out')
plt.savefig('dropout.pdf', bbox_inches='tight')

In [ ]:
data.CMs

In [ ]:
comb_cm_effects, new_names = combine_npi_samples(main_panel_dict, samples['alpha_i'], data.CMs)

In [ ]:
plt.figure(figsize=(4, 8), dpi=300)
plot_intervention_effectiveness(comb_cm_effects, new_names, xlim=[-10, 50], newfig=False)


In [ ]:
sigma_i = ((0.15 ** 2) / data.nCMs) ** 0.5

In [ ]:
for c_i, c in enumerate(data.unique_Cs):
    r_i = data.C_indices[c_i][0]
    comb_cm_effects, new_names = combine_npi_samples(grouped_dict, samples['alpha_ic'][:, r_i, :], data.CMs)
    plot_intervention_effectiveness(comb_cm_effects, new_names, xlim=[-50, 50])
    plt.title(c)

In [ ]:
plt.figure(figsize=(10, 3), dpi=300)
plt.subplot(121)

plt.scatter(np.arange(len(data.unique_Cs)), np.median(samples['psi_cases'], axis=0), marker='_', color='tab:blue')
li, lq, uq, ui = np.percentile(samples['psi_cases'], [2.5, 25, 75, 97.5], axis=0)
plt.plot([np.arange(len(data.unique_Cs)), np.arange(len(data.unique_Cs))], [li, ui], alpha=0.25, color='tab:blue')
plt.plot([np.arange(len(data.unique_Cs)), np.arange(len(data.unique_Cs))], [lq, uq], alpha=0.5, color='tab:blue')
plt.xticks(np.arange(len(data.unique_Cs)), data.unique_Cs)
plt.ylabel('$\Psi_C$. Higher is less noisy')

plt.subplot(122)
plt.scatter(np.arange(len(data.unique_Cs)), np.median(samples['psi_deaths'], axis=0), marker='_', color='tab:blue')
li, lq, uq, ui = np.percentile(samples['psi_deaths'], [2.5, 25, 75, 97.5], axis=0)
plt.plot([np.arange(len(data.unique_Cs)), np.arange(len(data.unique_Cs))], [li, ui], alpha=0.25, color='tab:blue')
plt.plot([np.arange(len(data.unique_Cs)), np.arange(len(data.unique_Cs))], [lq, uq], alpha=0.5, color='tab:blue')
plt.xticks(np.arange(len(data.unique_Cs)), data.unique_Cs)
plt.ylabel('$\Psi_D$. Higher is less noisy')
plt.suptitle('Output Noise Scales')
plt.tight_layout()

In [ ]:
np.mean(samples['psi_deaths'][:, 3])

In [ ]:
samples['psi_cases'][:, 3]

In [ ]:
from scipy import stats

nc = data.new_cases
nd = data.new_deaths

ec = np.array(samples['expected_cases'])
ed = np.array(samples['expected_deaths'])
nS, nRs, nDs = ec.shape

psi_c = (data.RC_mat @ samples['psi_cases'].T).T.reshape((nS, nRs, 1)).repeat(nDs, axis=2)
psi_d = (data.RC_mat @ samples['psi_deaths'].T).T.reshape((nS, nRs, 1)).repeat(nDs, axis=2)

cdf_c = stats.nbinom.cdf(nc.data, psi_c, psi_c/(psi_c+ec))
cdf_d = stats.nbinom.cdf(nd.data, psi_d, psi_d/(psi_d+ed))

In [ ]:
plt.figure(figsize=(7, 3), dpi=300)
plt.subplot(121)
sns.histplot(np.mean(cdf_c, axis=0)[np.logical_not(data.new_cases.mask)].flatten())
plt.title('PIT: Cases')

plt.subplot(122)
sns.histplot(np.mean(cdf_d, axis=0)[np.logical_not(data.new_deaths.mask)].flatten())
plt.title('PIT: Deaths')
plt.tight_layout()

In [ ]:
data.CMs

In [ ]:
cm_styles = {
    "Some Face-to-Face Businesses Closed": {
        "icon": "\uf07a",
        "color": "tab:orange"
    },
    "Gastronomy Closed": {
        "icon": "\uf2e7",
        "color": "k"
    },
    "Leisure Venues Closed": {
        "icon": "\uf000",
        "color": "k",
    },
    "Stay at Home Order AND All F2F Businesses Closed": {
        "icon": "\uf07a",
        "color": "tab:red",
    },
    "Curfew": {
        "icon": "\uf186",
        "color": "k",
    },
    "Childcare Closed": {
        "icon": "\uf77d",
        "color": "k",
    },
    "Primary Schools Closed": {
        "icon": "\uf51c",
        "color": "k",
    },
    "Secondary Schools Closed": {
        "icon": "\uf549",
        "color": "k",
    },
    "Universities Away": {
        "icon": "\uf19d",
        "color": "k",
    },
    "Public Indoor Gathering Person Limit - 1": {
        "icon": "\uf500",
        "color": (1, 0, 0),
    },
    "Public Indoor Gathering Person Limit - 2": {
        "icon": "\uf500",
        "color": (0.8, 0, 0),
    },
    "Public Indoor Gathering Person Limit - 6": {
        "icon": "\uf500",
        "color": (0.6, 0, 0),
    },
    "Public Indoor Gathering Person Limit - 30": {
        "icon": "\uf500",
        "color": (0.4, 0, 0),
    },
    "Private Indoor Gathering Person Limit - 1": {
        "icon": "\uf4b8",
        "color": (1, 0, 0),
    },
    "Private Indoor Gathering Person Limit - 2": {
        "icon": "\uf4b8",
        "color": (0.8, 0, 0),
    },
    "Private Indoor Gathering Person Limit - 6": {
        "icon": "\uf4b8",
        "color": (0.6, 0, 0),
    },
    "Private Indoor Gathering Person Limit - 30": {
        "icon": "\uf4b8",
        "color": (0.4, 0, 0),
    },
    "Mandatory Mask Wearing >= 3": {
        "icon": "\ue063",
        "color": "tab:blue",
    },
    "Extra Public Indoor Household Limit": {
        "icon": "\uf015",
        "color": "tab:green"
    },
    "Extra Private Indoor Household Limit": {
        "icon": "\uf015",
        "color": "tab:blue"
    },
}

In [ ]:
np.array(data.Rs)[data.C_indices[-2]]

In [ ]:
data.unique_Cs

In [ ]:
from epimodel.plotting.epicurve_plots import * 

In [ ]:
np.sum(ep.DPDv_pa[0, :28])

In [ ]:
np.sum(ep.DPCv_pa[0, :15])

In [ ]:
np.sum(ep.DPCv_pa[0, 7:])

In [ ]:
np.sum(ep.DPDv_pa[0, 17:])

In [ ]:
np.nonzero(np.cumsum(ep.DPCv_pa[0, :])>0.2)

In [ ]:
for r_i, r in enumerate(data.Rs):
    plot_area_summary(samples, r_i, data, cm_styles)
    plt.savefig(f'figs/{r_i}.pdf', bbox_inches='tight')

In [ ]:
data.Rs.index('Twente')

In [ ]:
data.nDs

In [ ]:
plt.plot(np.median(samples['Rt_walk'][:, 98, :], axis=0))
plt.plot(np.median(samples['Rt'][:, 98, :], axis=0))

In [ ]:
samples['Rt_walk'][:, 98, :].shape

In [ ]:
for r_i, r in enumerate(data.Rs):
    plot_area_summary(samples, r_i, data, cm_styles)
    plt.savefig(f'figs/{r_i}_w_outputnoise.pdf', bbox_inches='tight')

In [ ]:
samples['psi_cases'].shape